In [28]:
# Import California housing dataset from scikit-learn
from sklearn.datasets import fetch_california_housing

In [29]:
# Load the California housing dataset
data = fetch_california_housing()

# Extract features (X) and target variable (y) from the dataset
# X contains housing attributes like location, age, rooms, etc.
# y contains the median house value for California districts
X = data["data"]
y = data["target"]

In [30]:
# Import train_test_split function to divide data into training and testing sets
from sklearn.model_selection import train_test_split

# Split data into training (80%) and testing (20%) sets
# This helps evaluate how well our model generalizes to new, unseen data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [31]:
# Import regression models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Initialize a simple linear regression model
# Linear regression attempts to model the relationship between features and target by fitting a linear equation
reg = LinearRegression()

# Initialize a Random Forest regressor with parallel processing (-1 uses all available cores)
# Random Forest is an ensemble method that builds multiple decision trees and merges their predictions
# It generally performs better than linear regression for complex, non-linear relationships
forest = RandomForestRegressor(n_jobs=-1)

In [32]:
# Import StandardScaler for feature normalization
from sklearn.preprocessing import StandardScaler

# Initialize the scaler object
# Standardization transforms features to have zero mean and unit variance
# This is important for many ML algorithms, especially linear models
scaler = StandardScaler()

# Fit the scaler to the training data and transform it
# The fit_transform method learns the parameters from training data and applies the transformation
X_train = scaler.fit_transform(X_train)

# Transform the test data using the same scaling parameters
# Note: we only use transform() for test data to prevent data leakage.
# Data leakage occurs when information from outside the training dataset is used to create the model,
# leading to overly optimistic performance estimates. Here, we ensure the test data is transformed
# using the same scaler fitted on the training data to avoid introducing any information from the test set.
X_test = scaler.transform(X_test)

## Understanding Data Leakage in Machine Learning

Data leakage occurs when information from outside the training dataset is used to create the model, leading to unrealistically good performance metrics but poor real-world results.

### Simple Analogy: The Exam Cheater

Imagine a student named Alex who is studying for a math exam. The teacher has provided practice problems to help students prepare, but Alex secretly obtains a copy of the actual exam questions beforehand.

- Alex practices only the exact questions that will appear on the exam
- Alex scores perfectly on the test (100%)
- However, Alex hasn't actually learned how to solve math problems in general
- When faced with new problems in the next course, Alex fails miserably

This is exactly how data leakage works in machine learning:
- The model gets access to information it shouldn't have during training
- It performs exceptionally well on test data
- But it fails when deployed in the real world where that "leaked" information isn't available

### Real ML Example: Patient Readmission Prediction

Suppose we're building a model to predict which hospital patients will be readmitted within 30 days:

```python
# INCORRECT approach (with data leakage)
features = ['age', 'blood_pressure', 'discharge_medication', 'readmission_flag']
X = patient_data[features]
y = patient_data['was_readmitted']
# The 'readmission_flag' feature essentially tells the model the answer!

# CORRECT approach (no data leakage)
features = ['age', 'blood_pressure', 'discharge_medication']
X = patient_data[features]  # Only using information available at discharge time
y = patient_data['was_readmitted']

In [33]:
# Train the linear regression model on the scaled training data
# The fit method estimates the coefficients that minimize the prediction error
reg.fit(X_train, y_train)

# Train the Random Forest regressor on the scaled training data
# The fit method builds the ensemble of decision trees based on the training data
forest.fit(X_train, y_train)

RandomForestRegressor(n_jobs=-1)

# R² (Coefficient of Determination)

The R² coefficient is defined mathematically as:

$$R^2 = 1 - \frac{SS_{res}}{SS_{tot}}$$

where:

$$SS_{res} = \sum(y_i - \hat{y}_i)^2$$

$$SS_{tot} = \sum(y_i - \bar{y})^2$$

- **$SS_{res}$** is the sum of squares of residuals (errors) (variability in data the model cannot explain)
    > This measures how far off the predicted values ($\hat{y}_i$) are from the actual values ($y_i$). In other words, it captures the variability in the data that the model fails to explain.
- **$SS_{tot}$** is the total sum of squares which measures the variance of the observed values around their mean.
    > Basically, how far the actual values ($y_i$) are from the mean ($\bar{y}$). 

- $\frac{SS_{res}}{SS_{tot}}$ - Perentage of variability in the data that the model fails to explain. 
    - This fraction represents how much of the total variance in the dependent variable (target) is not explained by the model.

The fraction itself represents the percentage of variability the model cannot explain. So 1 minus that is the variability it can explain.

R² basically tells us how much (in percent) of the variance is explained by the model. We could consider this to be the accuracy metric of regression.

## Key Points About R²

- Easy to interpret as percentage of explained variance
- R² can be artificially inflated with useless variables
  - Adding new variables will always either do nothing or explain more
- Does not really tell us if the model is good or bad, model can have high R² but still large residuals
  - Explains most of the variance, but predictions still far off in absolute terms
- More suited for linear models (not so much for random forests, neural networks etc.)

## How Adding Variables Artificially Inflates R²

### Mathematical Explanation

When we add a new variable to a regression model, the sum of squared residuals (SSres) will either:
1. Decrease (if the variable has any correlation with the target)
2. Stay the same (if the variable has zero correlation)

Since R² = 1 - (SSres/SStot), and SStot remains constant, any decrease in SSres will increase R².

### Simple Example

Consider predicting house prices with these variables:
- Size (sq ft)
- Number of bedrooms

Let's say this model gives us R² = 0.65 (explains 65% of variance)

Now we add these additional variables:
- House number (completely random)
- Day of the week the house was listed (irrelevant)
- First letter of street name (meaningless)

Our R² might increase to 0.68, making it appear that our model improved, when in reality we've just added noise.

### Code Example


In [34]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Create sample data
np.random.seed(42)
n = 100
X_useful = pd.DataFrame({
    'size': np.random.normal(1500, 500, n),
    'bedrooms': np.random.randint(1, 6, n)
})

# True relationship plus noise
y = 100000 + 100 * X_useful['size'] + 15000 * X_useful['bedrooms'] + np.random.normal(0, 50000, n)

# Model with useful variables
model1 = LinearRegression()
model1.fit(X_useful, y)
y_pred1 = model1.predict(X_useful)
r2_1 = r2_score(y, y_pred1)

# Add meaningless variables
X_with_noise = X_useful.copy()
X_with_noise['house_number'] = np.random.randint(1, 10000, n)  # Random house numbers
X_with_noise['day_listed'] = np.random.randint(1, 8, n)  # Random day of week
X_with_noise['street_initial'] = np.random.randint(1, 27, n)  # Random letter

# Model with noise variables added
model2 = LinearRegression()
model2.fit(X_with_noise, y)
y_pred2 = model2.predict(X_with_noise)
r2_2 = r2_score(y, y_pred2)

print(f"R² with useful variables only: {r2_1:.4f}")
print(f"R² with noise variables added: {r2_2:.4f}")
print(f"R² inflation: {(r2_2 - r2_1):.4f}")

R² with useful variables only: 0.3983
R² with noise variables added: 0.4279
R² inflation: 0.0296


## Back to Linear regression vs Random forest

In [35]:
y_pred = reg.predict(X_test)

import numpy as np

# Calculate R² for Linear Regression model
# SSR_reg (Sum of Squared Residuals) measures the total squared prediction error
# SST_reg (Total Sum of Squares) measures the total variance in the actual data
SSR_reg = sum((y_test - y_pred) ** 2)
SST_reg = sum((y_test - np.mean(y_test)) ** 2)
R2_reg = 1 - (SSR_reg / SST_reg)

# Output R² for Linear Regression
R2_reg

0.5993358188700904

In [36]:
# Generate predictions using the Random Forest model
y_pred = forest.predict(X_test)

# Calculate R² for Random Forest model using the same method
SSR_forest = sum((y_test - y_pred) ** 2)
SST_forest = sum((y_test - np.mean(y_test)) ** 2)
R2_forest = 1 - (SSR_forest / SST_forest)

# Output R² for Random Forest
R2_forest
# 0.8088256814736522

0.8151885673631656

# Adjusted R²

The Adjusted R² statistic modifies the standard R² to account for the number of predictors in the model:

$$R^2_{adj} = 1 - \frac{(1 - R^2)(n - 1)}{n - p - 1} = 1 - (1 - R^2) \cdot \frac{n-1}{n-p-1}$$

Where:
- n = number of observations
- p = number of predictors (independent variables)

## Key Points About Adjusted R²

- Adjusts R² for number of predictors, penalizing unnecessary complexity
- Better for comparing models with different number of variables (will only improve with useful variables)
- Use R² and Adj. R² in combination with other metrics
- Also more suited for linear models (like R²)

### Implementation


In [ ]:
from sklearn.metrics import r2_score

def adjusted_r2(r2, n, p):
    """
    Calculate adjusted R-squared

    Parameters:
    r2 (float): R-squared value
    n (int): Number of observations
    p (int): Number of predictors

    Returns:
    float: Adjusted R-squared value
    """
    return 1 - (1 - r2) * (n - 1) / (n - p - 1)

# Example usage
r2 = r2_score(y_test, y_pred)
adj_r2 = adjusted_r2(r2, len(y_test), X_test.shape[1])
print(f"R²: {r2:.4f}, Adjusted R²: {adj_r2:.4f}")

R²: 0.8152, Adjusted R²: 0.8148


# MSE (Mean Squared Error)

Mean Squared Error is a measure of the average squared difference between the predicted values and the actual values:

$$\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$$

Where:
- $y_i$ = actual value
- $\hat{y}_i$ = predicted value
- $n$ = number of observations

## Key Characteristics of MSE

- Not very intuitive, RMSE more intuitive
- Emphasizes larger errors due to squaring, useful when large errors are unacceptable
- Differentiable, useful in optimization algorithms
- Good when trying to minimize overall error in the model
- Highly sensitive to outliers
- Scale-dependent

In [43]:
# Import mean_squared_error from sklearn.metrics
from sklearn.metrics import mean_squared_error

n = X_test.shape[0]  # Number of observations in the test set

# Generate predictions using the Linear Regression model
y_pred = reg.predict(X_test)
mse_reg = mean_squared_error(y_test, y_pred)
# Calculate MSE for Linear Regression manually: MSE = (1/n) * sum((actual - predicted)²)
mse_reg2 = (1/n) * sum((y_test - y_pred)**2)
print(mse_reg)
print(mse_reg2)
print()

# Generate predictions using the Random Forest model
y_pred = forest.predict(X_test)
mse_forest = mean_squared_error(y_test, y_pred)
# Calculate MSE for Random Forest manually: MSE = (1/n) * sum((actual - predicted)²)
mse_forest2 = (1/n) * sum((y_test - y_pred)**2)
print(mse_forest)
print(mse_forest2)

0.5296234193188425
0.5296234193188414

0.24429551602616947
0.24429551602616947


# RMSE (Root Mean Squared Error)

Root Mean Squared Error is the square root of the Mean Squared Error, giving a metric that has the same units as the target variable:

$$\text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}$$

Where:
- $y_i$ = actual value
- $\hat{y}_i$ = predicted value
- $n$ = number of observations

## Key Characteristics of RMSE

- Same units as response variables, easy to interpret
- Penalizes larger errors due to squaring
- Good when trying to minimize overall error in the model
- Sensitive to outliers

### Implementation

In [44]:
from sklearn.metrics import root_mean_squared_error

# Generate predictions using the Linear Regression model
y_pred = reg.predict(X_test)
# Calculate RMSE for Linear Regression using sklearn function
rmse_reg = root_mean_squared_error(y_test, y_pred)
# Calculate RMSE for Linear Regression manually
rmse_reg2 = np.sqrt((1/n) * sum((y_test - y_pred)**2))
print(rmse_reg)
print(rmse_reg2)
print()


# Generate predictions using the Random Forest model
y_pred = forest.predict(X_test)
# Calculate RMSE for Random Forest using sklearn function
rmse_forest = root_mean_squared_error(y_test, y_pred)
# Calculate RMSE for Random Forest manually
rmse_forest2 = np.sqrt((1/n) * sum((y_test - y_pred)**2))
print(rmse_forest)
print(rmse_forest2)

0.7277523062957908
0.72775230629579

0.49426259824729757
0.49426259824729757
